In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import random as rnd

# visualization
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

from sklearn.impute import KNNImputer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Adding the data

In [2]:
trainData = pd.read_csv('./kaggle/input/spaceship-titanic/train.csv')
testData = pd.read_csv('./kaggle/input/spaceship-titanic/test.csv')
combine = [trainData, testData]

trainData.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
trainData.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [4]:
trainData.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

Drop features we dont want to use

In [5]:
trainData = trainData.drop(['Name'], axis=1)
testData = testData.drop(['Name'], axis=1)
combine = [trainData, testData]

In [6]:
trainData.HomePlanet.mode()
trainData[['HomePlanet', 'Transported']].groupby(['HomePlanet'], as_index=False).mean().sort_values(by='HomePlanet', ascending=False)

,HomePlanet,Transported
2,Mars,0.523024
1,Europa,0.658846
0,Earth,0.423946


In [7]:
trainData['HomePlanet'] = trainData['HomePlanet'].fillna('Earth')
testData['HomePlanet'] = testData['HomePlanet'].fillna('Earth')


trainData['HomePlanet'] = trainData['HomePlanet'].map({'Earth':0, 'Mars':1, 'Europa':2}).astype(int)
testData['HomePlanet'] = testData['HomePlanet'].map({'Earth':0, 'Mars':1, 'Europa':2}).astype(int)
combine = [trainData, testData]

trainData.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,2,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,0,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,2,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,2,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,0,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [8]:
trainData['RoomService'].fillna(0,inplace=True)
testData['RoomService'].fillna(0,inplace=True)

trainData['FoodCourt'].fillna(0,inplace=True)
testData['FoodCourt'].fillna(0,inplace=True)

trainData['ShoppingMall'].fillna(0,inplace=True)
testData['ShoppingMall'].fillna(0,inplace=True)

trainData['Spa'].fillna(0,inplace=True)
testData['Spa'].fillna(0,inplace=True)

trainData['VRDeck'].fillna(0,inplace=True)
testData['VRDeck'].fillna(0,inplace=True)

trainData.isnull().sum()



PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [9]:
testData['VIP'].fillna(value=False,inplace=True)
trainData['VIP'].fillna(value=False,inplace=True)

testData['Destination'].fillna(value='TRAPPIST-1e',inplace=True)
trainData['Destination'].fillna(value='TRAPPIST-1e',inplace=True)

testData['Age'].fillna(value=testData.Age.mean(),inplace=True)
trainData['Age'].fillna(value=trainData.Age.mean(),inplace=True)


trainData.isnull().sum()


PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [10]:
testData['Cabin'].fillna(value='X/X/X',inplace=True)
trainData['Cabin'].fillna(value='X/X/X',inplace=True)

In [11]:
newTest = testData['Cabin'].str.split('/',n=2,expand = True)
newTrain = trainData['Cabin'].str.split('/',n=2,expand = True)

testData['deck'] = newTest[0]
trainData['deck'] = newTrain[0]

testData['side'] = newTest[2]
trainData['side'] = newTrain[2]

testData.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,side
0,0013_01,0,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,S
1,0018_01,0,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,S
2,0019_01,2,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,S
3,0021_01,2,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,S
4,0023_01,0,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,S


In [12]:
trainData = trainData.drop(['Cabin'], axis=1)
testData = testData.drop(['Cabin'], axis=1)

In [13]:
print(trainData.deck.mode())
print(trainData.side.mode())

0    F
Name: deck, dtype: object
0    S
Name: side, dtype: object


In [14]:
trainData['deck'] = trainData['deck'].map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7, 'X':5}).astype(int)
testData['deck'] = testData['deck'].map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7, 'X':5}).astype(int)

trainData['side'] = trainData['side'].map({'P':0, 'S':1, 'X':1}).astype(int)
testData['side'] = testData['side'].map({'P':0, 'S':1, 'X':1}).astype(int)

In [15]:
print('trainData')
print(trainData.head())
print('testData')
print(testData.head())

trainData
  PassengerId  HomePlanet CryoSleep  Destination   Age    VIP  RoomService  \
0     0001_01           2     False  TRAPPIST-1e  39.0  False          0.0   
1     0002_01           0     False  TRAPPIST-1e  24.0  False        109.0   
2     0003_01           2     False  TRAPPIST-1e  58.0   True         43.0   
3     0003_02           2     False  TRAPPIST-1e  33.0  False          0.0   
4     0004_01           0     False  TRAPPIST-1e  16.0  False        303.0   

   FoodCourt  ShoppingMall     Spa  VRDeck  Transported  deck  side  
0        0.0           0.0     0.0     0.0        False     1     0  
1        9.0          25.0   549.0    44.0         True     5     1  
2     3576.0           0.0  6715.0    49.0        False     0     1  
3     1283.0         371.0  3329.0   193.0        False     0     1  
4       70.0         151.0   565.0     2.0         True     5     1  
testData
  PassengerId  HomePlanet CryoSleep  Destination   Age    VIP  RoomService  \
0     0013_01 

In [16]:
trainData['Destination'] = trainData['Destination'].map({'TRAPPIST-1e':0, '55 Cancri e':1, 'PSO J318.5-22':2}).astype(int)
testData['Destination'] = testData['Destination'].map({'TRAPPIST-1e':0, '55 Cancri e':1, 'PSO J318.5-22':2}).astype(int)

In [17]:
trainData.isnull().sum()


PassengerId       0
HomePlanet        0
CryoSleep       217
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
deck              0
side              0
dtype: int64

In [18]:
ammenities = ['RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck']
testData['SpentMoney'] = testData[ammenities].sum(axis=1)
trainData['SpentMoney'] = trainData[ammenities].sum(axis=1)

In [19]:
trainData = trainData.drop(['RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck'], axis=1)
testData = testData.drop(['RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck'], axis=1)

Using a KNNImputer to fill missing values of age from Fare and Sibsp features

In [20]:
trainData.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,deck,side,SpentMoney
0,0001_01,2,False,0,39.0,False,False,1,0,0.0
1,0002_01,0,False,0,24.0,False,True,5,1,736.0
2,0003_01,2,False,0,58.0,True,False,0,1,10383.0
3,0003_02,2,False,0,33.0,False,False,0,1,5176.0
4,0004_01,0,False,0,16.0,False,True,5,1,1091.0


In [21]:
knnimputer = KNNImputer()

impTrainData = knnimputer.fit_transform(trainData)
impTestData = knnimputer.fit_transform(testData)

trainData['CryoSleep'] = impTrainData[:,2]
testData['CryoSleep'] = impTestData[:,2]

In [22]:
testData.isnull().sum()

PassengerId    0
HomePlanet     0
CryoSleep      0
Destination    0
Age            0
VIP            0
deck           0
side           0
SpentMoney     0
dtype: int64

Normalizing the Fare feature

In [23]:
X_train = trainData.drop(['Transported','PassengerId'],axis = 1)
Y_train = trainData["Transported"]
X_test  = testData.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((8693, 8), (8693,), (4277, 8))

In [24]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,deck,side,SpentMoney
0,2,0.0,0,39.0,False,1,0,0.0
1,0,0.0,0,24.0,False,5,1,736.0
2,2,0.0,0,58.0,True,0,1,10383.0
3,2,0.0,0,33.0,False,0,1,5176.0
4,0,0.0,0,16.0,False,5,1,1091.0


In [25]:
X_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,deck,side,SpentMoney
0,0,1.0,0,27.0,False,6,1,0.0
1,0,0.0,0,19.0,False,5,1,2832.0
2,2,1.0,1,31.0,False,2,1,0.0
3,2,0.0,0,38.0,False,2,1,7418.0
4,0,0.0,0,20.0,False,5,1,645.0


In [26]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc


73.44

This cell finds the best number of neighbors to use for our model

In [27]:
parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(KNeighborsClassifier(), parameters)
gridsearch.fit(X_train, Y_train)

gridsearch.best_params_

{'n_neighbors': 41}

In [28]:
knn = KNeighborsClassifier(n_neighbors = 41)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
knn.score(X_train, Y_train)

0.7409409870010353

In [29]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

94.24

In [30]:
# parameters = {"n_estimators":[1000,2500, 5000] ,"max_depth": [3,5,7,8,10], "min_samples_split": [5,8,10]}
# gridsearch = GridSearchCV(RandomForestClassifier(), parameters)
# gridsearch.fit(X_train, Y_train)

# gridsearch.best_params_
# # {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 1000}

In [31]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=1000, min_samples_split = 5, max_depth=10)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

78.24

In [32]:
gbm = xgb.XGBClassifier(learning_rate = .15, max_depth=7,subsample = .7)
gbm.fit(X_train, Y_train)
gmbScore = round(gbm.score(X_train, Y_train) *100, 2)
gmbScore

83.78

In [33]:
models = pd.DataFrame({
    'Model': [ 'KNN','Random Forest','Decision Tree', 'XGB'],
    'Score': [acc_knn,acc_random_forest,acc_decision_tree,gmbScore]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Decision Tree,94.24
3,XGB,83.78
1,Random Forest,78.24
0,KNN,74.09


In [1]:
Y_pred = gbm.predict(X_test)

NameError: name 'gbm' is not defined

In [35]:
submission = pd.DataFrame({
        "PassengerId": testData["PassengerId"],
        "Survived": Y_pred
    })

submission.to_csv('/kaggle/working/submission.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/kaggle/working'